<a href="https://colab.research.google.com/github/jimin0/Graduation_Project_HE_Study/blob/main/ex4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 졸업작품
>12/06 미팅 준비

## 동형암호로 연산으로 바꾸기 (합성곱 신경망(CNN)에서 사용되는 연산)

```

       for i in range(output_height):
            for j in range(output_width):
                for f in range(self.num_filters):
                    h_start = i * self.stride
                    h_end = h_start + self.kernel_size
                    w_start = j * self.stride
                    w_end = w_start + self.kernel_size
                    receptive_field = input_volume[:, h_start:h_end, w_start:w_end, :]
                    ###     ###
                    output_volume[:, i, j, f] = np.sum(receptive_field * self.kernel[:, :, :, f], axis=(1, 2, 3)) + self.bias[:,:,:,f]
        

```


교체부분

`output_volume[:, i, j, f] = np.sum(receptive_field * self.kernel[:, :, :, f], axis=(1, 2, 3)) + self.bias[:,:,:,f]`


low level
```
                    for a in range(self.batch_size):
                        temp = 0
                        for b in range(self.kernel_size):
                            for c in range(self.kernel_size):
                                for d in range(self.input_channels):
                                    temp += receptive_field[a,b,c,d] * self.kernel[b,c,d,f]

                        output_volume[a, i, j, f] = temp + self.bias[:,:,:,f]
```

### 이해를 위한..  : Don't execution

In [ ]:
# 시뮬레이션을 위해 가정 - 이해를 위한..

batch_size = 4
input_channels = 3
kernel_size = 3

for a in range(batch_size):           # batch_size = 4, 4개의 데이터 샘플 처리
  temp = 0                            # 합성곱 연산 결과 저장할 임시 변수
  for b in range(kernel_size):        # kernel_size = 3, 3*3 윈도우를 의미
    for c in range(kernel_size):
      for d in range(input_channels):  # input_channels =3,  입력한 채널 수 만큼 반복 -> RGB 이미지의 각 색상 채널(R,G,B)
          temp += receptive_field[a,b,c,d] * self.kernel[b,c,d,f] # 현재 처리 중인 데이터 샘플(수용필드) * 해당위치와 채널에서의 커널값 (내적)


      output_volume[a, i, j, f] = temp + self.bias[:,:,:,f] #temp+bias

### 동형암호 적용
>곱셈 한번, 덧셈 두번?

#### 환경설정

In [ ]:
!git clone https://github.com/Huelse/SEAL-Python.git
%cd SEAL-Python

# Install dependencies
!pip3 install numpy pybind11

# Init the SEAL and pybind11
!git submodule update --init --recursive
# Get the newest repositories (dev only)
# git submodule update --remote

# Build the SEAL lib
%cd SEAL
!cmake -S . -B build -DSEAL_USE_MSGSL=OFF -DSEAL_USE_ZLIB=OFF
!cmake --build build
%cd ..

# Run the setup.py
!python3 setup.py build_ext -i


Cloning into 'SEAL-Python'...
remote: Enumerating objects: 1628, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 1628 (delta 176), reused 191 (delta 162), pack-reused 1376
Receiving objects: 100% (1628/1628), 8.68 MiB | 14.86 MiB/s, done.
Resolving deltas: 100% (882/882), done.
/content/SEAL-Python/SEAL-Python/SEAL-Python
Submodule 'SEAL' (https://github.com/microsoft/SEAL.git) registered for path 'SEAL'
Submodule 'pybind11' (https://github.com/pybind/pybind11.git) registered for path 'pybind11'
Cloning into '/content/SEAL-Python/SEAL-Python/SEAL-Python/SEAL'...
Cloning into '/content/SEAL-Python/SEAL-Python/SEAL-Python/pybind11'...
Submodule path 'SEAL': checked out '206648d0e4634e5c61dcf9370676630268290b59'
Submodule path 'pybind11': checked out 'be97c5a98b4b252c524566f508b5c79410d118c6'
/content/SEAL-Python/SEAL-Python/SEAL-Python/SEAL
-- Build type (CMAKE_BUILD_TYPE): Release
-- The CXX compiler identification is 

#### 객체 생성 및 키 생성



In [ ]:
# SEAL 라이브러리의 모든 클래스와 함수를 임포트합니다.
from seal import *
import numpy as np

# BGV 암호화 스킴을 사용할 암호화 매개변수 객체를 생성합니다.
parms = EncryptionParameters(scheme_type.bgv)

"""
poly_modulus_degree가 클수록 암호화된 텍스트의 크기가 커지고 모든 연산이 느려지지만,
더 복잡한 암호화된 계산을 가능하게 함. 권장되는 값들은 1024, 2048, 4096, 8192, 16384, 32768이지만,
이 범위를 넘어가는 것도 가능
"""
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 20))


context = SEALContext(parms)


# 계수 모듈러스를 설정합니다. 이는 암호화 시 사용되는 모듈러스 값으로,
# BFVDefault는 보안 수준에 맞는 기본 모듈러스 값을 선택합니다.
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))

# 평문 모듈러스를 설정합니다.
# 이 모듈러스는 암호화되지 않은 데이터가 취할 수 있는 값의 범위를 정의합니다.
# 이 경우, 값의 범위는 0에서 255입니다.
parms.set_plain_modulus(256)


# 키 생성

keygen = KeyGenerator(context)

secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
galois_keys =  keygen.create_galois_keys()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)


batch_encoder = BatchEncoder(context)
slot_count = batch_encoder.slot_count()
row_size = slot_count / 2
print(f'Plaintext matrix row size: {row_size}')

Plaintext matrix row size: 4096.0


#### 암호화

시뮬레이션을 위해 임의의 값 할당

In [ ]:
# 예시 값을 설정
batch_size = 4
kernel_size = 3
input_channels = 3

a, b, c, d = 0, 0, 0, 0

f = 0  # 출력 채널의 인덱스 예시
i, j = 0, 0  # output_volume의 인덱스 예시
temp =0

# receptive_field, kernel, bias, output_volume의 차원 설정
receptive_field = np.random.randint(0, 5, (batch_size, kernel_size, kernel_size, input_channels))
kernel = np.random.randint(0, 5, (kernel_size, kernel_size, input_channels, batch_size))
bias = np.random.randint(0, 5, (1, 1, 1, batch_size))
output_volume = np.zeros((batch_size, 1, 1, batch_size))  # output_volume 초기화

# 결과 출력
#receptive_field, kernel, bias, output_volume

print("선언(Beforeconv layer 연산)")
print("receptive_fields[a,b,c,d] = ",receptive_field[a,b,c,d])
print("kernel[b,c,d,f]  = ",kernel[b,c,d,f] )
print("bias[0, 0, 0, f]  = ",bias[0, 0, 0, f] )
print("temp = 0")
print("receptive_field[a,b,c,d] * self.kernel[b,c,d,f] = ",receptive_field[a,b,c,d]*kernel[b,c,d,f])
print("output_volume[a, i, j, f] = temp + bias[0, 0, 0, f] = ",temp + bias[0, 0, 0, f])

print()

# 합성곱 연산 수행
for a in range(batch_size):
    temp = 0
    for b in range(kernel_size):
        for c in range(kernel_size):
            for d in range(input_channels):
                temp += receptive_field[a, b, c, d] * kernel[b, c, d, f]

    output_volume[a, i, j, f] = temp + bias[0, 0, 0, f]




print("=============================================================================================")
print("(After conv layer 연산 끝난 후)")
print("receptive_fields[a,b,c,d] = ",receptive_field[a,b,c,d])
print("kernel[b,c,d,f]  = ",kernel[b,c,d,f] )
print("bias[0, 0, 0, f]  = ",bias[0, 0, 0, f] )
print("receptive_field[a,b,c,d] * self.kernel[b,c,d,f] = ",receptive_field[a,b,c,d]*kernel[b,c,d,f])
print("output_volume[a, i, j, f] = temp + bias[0, 0, 0, f] = ",temp + bias[0, 0, 0, f])
print("result temp = ",temp)
print("result output_volume = ",output_volume[a, i, j, f])

선언(Beforeconv layer 연산)
receptive_fields[a,b,c,d] =  1
kernel[b,c,d,f]  =  3
bias[0, 0, 0, f]  =  3
temp = 0
receptive_field[a,b,c,d] * self.kernel[b,c,d,f] =  3
output_volume[a, i, j, f] = temp + bias[0, 0, 0, f] =  3

(After conv layer 연산 끝난 후)
receptive_fields[a,b,c,d] =  2
kernel[b,c,d,f]  =  2
bias[0, 0, 0, f]  =  3
receptive_field[a,b,c,d] * self.kernel[b,c,d,f] =  4
output_volume[a, i, j, f] = temp + bias[0, 0, 0, f] =  111
result temp =  108
result output_volume =  111.0


###### 폐기 코드
>----> matrix에 넣어서 한꺼번에 plain text, cipertext로 변환 아이디어 -> Chipertext 클래스는 암호문 배열 인덱싱 계산이 없음... --> 폐기..

In [ ]:
# 암호화 : receptive_field, self.kernel, self.bias
pod_matrix = [0] * slot_count


"""Q. 질문 int로 변환해도 되는지, string으로 해야하는지, 실수형으로 인자는 못 받음.  -> 일단 Int로 전환"""
receptive_field_value = int(receptive_field[a,b,c,d])
kernel_value = int(kernel[b,c,d,f])   # 일단 self 지움.  -> self.bias
bias_value = int(bias[0, 0, 0, f])       # 일단 self 지움.  -> self.bias

pod_matrix[0] = receptive_field_value
pod_matrix[1] = kernel_value
pod_matrix[2] = bias_value

# 값이 할당 잘 되어있는지 확인
print("pod_matrix = ",pod_matrix[:3])

x_plain = batch_encoder.encode(pod_matrix[:3])  # plain text로 변환
print(x_plain)

x_encrypted = encryptor.encrypt(x_plain)
print(x_encrypted)

print(f'noise budget in freshly encrypted x: {decryptor.invariant_noise_budget(x_encrypted)}')
print('-'*50)

pod_matrix =  [0, 3, 1]
noise budget in freshly encrypted x: 146
--------------------------------------------------


##### here!

모든 변수 평문으로 인코딩하고 암호화 (type = int)

In [ ]:
receptive_field_value_int = int(receptive_field[a,b,c,d])
kernel_value_int = int(kernel[b,c,d,f])         # 일단 self 지움.  -> self.bias
bias_value_int = int(bias[0, 0, 0, f])                      # 일단 self 지움.  -> self.bias
output_volume_int = int(output_volume[a, i, j, f])

print("receptive_field_value = ",receptive_field_value_int)
print("kernel_value = ",kernel_value_int)
print("bias_value = ",bias_value_int)
print("output_volume = ",output_volume_int)

# 평문으로 인코딩하고 암호화 (int)
receptive_field_int_pt = Plaintext(receptive_field_value_int)
receptive_field_int_encrypted = Ciphertext()
receptive_field_int_encrypted = encryptor.encrypt(receptive_field_int_pt)

kernel_int_pt = Plaintext(kernel_value_int)
kernel_int_encrypted = Ciphertext()
kernel_int_encrypted = encryptor.encrypt(kernel_int_pt)

"Q. 질문 : bias 도 chiphertext로 해야하는지, 아님 plaintext인지"
bias_int_pt = Plaintext(bias_value_int)
bias_int_encrypted = Ciphertext()
bias_int_encrypted = encryptor.encrypt(bias_int_pt)


output_volume_int_pt=Plaintext(output_volume_int)
output_volume_int_encrypted = Ciphertext()
output_volume_int_encrypted = encryptor.encrypt(output_volume_int_pt)

print()
(receptive_field_int_encrypted, kernel_int_encrypted, bias_int_encrypted, output_volume_int_encrypted)

receptive_field_value =  2
kernel_value =  2
bias_value =  3
output_volume =  111



(<seal.Ciphertext at 0x7d1a09cd4cf0>,
 <seal.Ciphertext at 0x7d1a04645e30>)

모든 변수 평문으로 인코딩하고 암호화 (type = string)

In [ ]:
receptive_field_value = int(receptive_field[a, b, c, d])
kernel_value = int(kernel[b, c, d, f])
bias_value = int(bias[0, 0, 0, f])
output_volume_value = int(output_volume[a, i, j, f])


print("receptive_field_value = ",receptive_field_value)
print("kernel_value = ",kernel_value)
print("bias_value = ",bias_value)
print("output_volume = ",output_volume)

# 평문으로 인코딩하고 암호화 (str)
receptive_field_str_pt = Plaintext(str(receptive_field_value))
receptive_str_field_encrypted = encryptor.encrypt(receptive_field_str_pt)

kernel_str_pt = Plaintext(str(kernel_value))
kernel_str_encrypted = encryptor.encrypt(kernel_str_pt)

bias_str_pt = Plaintext(str(bias_value))
bias_str_encrypted = encryptor.encrypt(bias_str_pt)

output_volume_str_pt = Plaintext(str(output_volume_value))
output_volume_str_encrypted = encryptor.encrypt(output_volume_str_pt)

print()
(receptive_str_field_encrypted, kernel_str_encrypted, bias_str_encrypted, output_volume_str_encrypted)

receptive_field_value =  2
kernel_value =  2
bias_value =  3
output_volume =  111



(<seal.Ciphertext at 0x7d1a046469f0>,
 <seal.Ciphertext at 0x7d1a04647ab0>)

중간 점검 : 복호화 테스트

In [ ]:
# receptive_field 복호화 test (int)
decrypted_receptive_field_int_pt = Plaintext()
decryptor.decrypt(receptive_str_field_encrypted, decrypted_receptive_field_int_pt)
decrypted_receptive_field_int_value = decrypted_receptive_field_int_pt.to_string()

print(decrypted_receptive_field_int_value)


# receptive_field 복호화 test (string)
decrypted_receptive_field_str_pt = Plaintext()
decryptor.decrypt(receptive_str_field_encrypted, decrypted_receptive_field_str_pt)
decrypted_receptive_field_value = decrypted_receptive_field_str_pt.to_string()

print(decrypted_receptive_field_value)


2
2


##### 그냥 참고코드 -> 폐기

In [ ]:
string = "HELLO"
encoded_string = [ord(char) for char in string]  # 문자열을 ASCII 코드로 변환

encrypted_values = []
for value in encoded_string:
    plaintext = Plaintext(str(value))
    encrypted_value = encryptor.encrypt(plaintext)
    encrypted_values.append(encrypted_value)

print(encrypted_values[0])

##### main

연산 int 형으로

In [ ]:
# receptive_field[a,b,c,d] * self.kernel[b,c,d,f]

product = evaluator.multiply(receptive_field_int_encrypted, kernel_int_encrypted) # 4= 2*2

# 복호화 및 결과 확인
decrypted_product_int_pt = Plaintext()
decryptor.decrypt(product, decrypted_product_int_pt)
decrypted_product_value = decrypted_product_int_pt.to_string()

print(decrypted_product_value)
# 2

0


연산 string으로

In [ ]:
# receptive_field[a,b,c,d] * self.kernel[b,c,d,f]

product = evaluator.multiply(receptive_str_field_encrypted, kernel_str_encrypted)  # 4= 2*2

# 복호화 및 결과 확인
decrypted_product_str_pt = Plaintext()
decryptor.decrypt(product, decrypted_product_str_pt)
decrypted_product_value = decrypted_product_str_pt.to_string()

print(decrypted_product_value)

4


In [ ]:
# temp += receptive_field[a,b,c,d] * self.kernel[b,c,d,f]
temp_encrypted = encryptor.encrypt(Plaintext("0"))  # 암호화된 0으로 temp 초기화
temp_encrypted = evaluator.add(temp_encrypted, product)

# 복호화 및 결과 확인
decrypted = Plaintext()
decryptor.decrypt(temp_encrypted, decrypted)
decrypted.to_string()

print(decrypted.to_string())

4


In [ ]:
# output_volume[a, i, j, f] = temp + self.bias[:,:,:,f] #temp+bias

output_volume_encrypted =  evaluator.add(temp_encrypted, bias_encrypted)

##### conv layer 연산 종합코드

In [ ]:
# 가정: 동형암호 라이브러리의 곱셈과 덧셈 연산 함수
# multiply_encrypted, add_encrypted는 암호화된 데이터에 대해 연산을 수행하는 함수

for a in range(batch_size):
    temp_encrypted = encryptor.encrypt(Plaintext("0"))  # 암호화된 0으로 temp 초기화

    for b in range(kernel_size):
        for c in range(kernel_size):
            for d in range(input_channels):
                # 암호화된 receptive_field와 kernel의 곱셈
                product = evaluator.multiply(receptive_field_encrypted, kernel_encrypted)
                # 곱셈 결과를 temp에 누적
                temp_encrypted = evaluator.add(temp_encrypted, product)

    # 암호화된 bias를 temp에 더함
    output_volume_encrypted =  evaluator.add(temp_encrypted, bias_encrypted)


In [ ]:
output_volume_encrypted

#### 복호화

In [ ]:
# 복호화 및 결과 확인
decrypted = Plaintext()
decryptor.decrypt(output_volume_encrypted, decrypted)
decrypted.to_string()

print(decrypted.to_string())

0


In [ ]:
# 복호화 및 결과 확인
decrypted = Plaintext()
decryptor.decrypt(output_volume_encrypted, decrypted)




# 결과 확인
expected_result = 2 * 3 * 4 * 2 * 3 * 1
if decoded_result[0] == expected_result:
    print(f'Correct!: [Expected: {expected_result}] == [Decrypted: {decoded_result[0]}]')
else:
    print(f'Incorrect!: [Expected: {expected_result}] != [Decrypted: {decoded_result[0]}]')